In [100]:
from gurobipy import Model, GRB
import numpy as np
import math

In [101]:
current_inventory = {'1': 200,
                    '2': 10,
                    '3': 120,
                    '4': 120,
                    '5': 70,
                    '6': 120,
                    '7': 50,
                    '8': 120,
                    '9': 10}

In [102]:
current_day = 1
num_stores = 9

In [103]:
daily_demands = {'1': [(83,5.8), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '2': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '3': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '4': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '5': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '6': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '7': [(20,5), (15,7), (20,4), (25,6), (10,6), (8,5), (20,4)],
                '8': [(20,5), (15,7), (20,4), (25,6), (10,6), (6,2), (20,4)],
                '9': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)]}

In [104]:
costs = {'stockout': 30,
        'holding': 1,
        'transshipment': 1}

In [105]:
def convolution(store_id, start_day):
    
    store_data = daily_demands[store_id]
    future_data = store_data[start_day:]
    mean_values = [mean for mean, _ in future_data]
    std_dev_values = [std_dev for _, std_dev in future_data]
    convoluted_mean = sum(mean_values)
    convoluted_st_dev = math.sqrt(sum(x**2 for x in std_dev_values))
    return convoluted_mean, convoluted_st_dev

In [106]:
convolution('2', current_day)

(100, 14.212670403551895)

In [107]:
ratio = (costs['stockout'] - costs['transshipment']) / (costs['stockout'] + costs['holding'] - costs['transshipment'])
ratio

0.9666666666666667

In [108]:
def update_inventory(minus):

    for store in current_inventory:
        current_inventory[store] -= minus.get(store, 0)

In [62]:
todays_sales = {'1': 10,
                    '2': 50,
                    '3': 20,
                    '4': 20,
                    '5': 20,
                    '6': 20,
                    '7': 20,
                    '8': 20,
                    '9': 20}

In [64]:
update_inventory(todays_sales)

In [109]:
current_inventory

{'1': 200,
 '2': 10,
 '3': 120,
 '4': 120,
 '5': 70,
 '6': 120,
 '7': 50,
 '8': 120,
 '9': 10}

In [122]:
shipper_cand = []
def shipper_candidates(current_inventory, current_day):
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = mean + (ratio*st_dev)
        if critical_point < current_inventory[i]:
            print(f"for store {i} critical point, current inventory: {critical_point, current_inventory[i]}")
            shipper_cand.append(i)
    return shipper_cand

In [123]:
shipper_candidates(current_inventory, current_day)

for store 1 critical point, current inventory: (113.7389147234335, 200)
for store 3 critical point, current inventory: (113.7389147234335, 120)
for store 4 critical point, current inventory: (113.7389147234335, 120)
for store 6 critical point, current inventory: (113.7389147234335, 120)
for store 8 critical point, current inventory: (108.11229861660361, 120)


['1', '3', '4', '6', '8']

In [124]:
receiving_cand = []
def receiving_candidates(current_inventory, current_day):
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = mean - (ratio*st_dev)
        if critical_point > current_inventory[i]:
            print(f"for store {i} critical point, current inventory: {critical_point, current_inventory[i]}")
            receiving_cand.append(i)
    return receiving_cand

In [125]:
receiving_candidates(current_inventory, current_day)

for store 2 critical point, current inventory: (86.2610852765665, 10)
for store 5 critical point, current inventory: (86.2610852765665, 70)
for store 7 critical point, current inventory: (85.10305807134455, 50)
for store 9 critical point, current inventory: (86.2610852765665, 10)


['2', '5', '7', '9']

In [114]:
model = Model("Lateral Transshipments")

Restricted license - for non-production use only - expires 2025-11-24


In [129]:
shippers = list(map(int, shipper_cand))
receivers = list(map(int, receiving_cand))

In [126]:
x = model.addVars(list(map(int, shipper_cand)), list(map(int, receiving_cand)), lb=0, vtype=GRB.CONTINUOUS, name='x')

In [128]:
# nested for loop
model.setObjective(
    sum(costs['stockout'] * max(0, sum(d[0] for d in daily_demands[j]) - (current_inventory[j] + sum(x[i, j] for i in shippers))) for j in receivers) +
    sum(costs['transshipment'] * x[i, j] for i in shippers for j in receivers),
    GRB.MINIMIZE
)

<class 'list'>
<class 'function'>
